In [413]:
from glob import glob
import itertools
import csv

In [2]:
import jamotools

In [3]:
'''
Hangul Jamo: Consist of Choseong, Jungseong, Jongseong. It is divided mordern Hangul and old Hangul that does not use in nowadays. Jamotools supports modern Hangul Jamo area.
1100 ~ 1112 (Choseong)
1161 ~ 1175 (Jungseong)
11A8 ~ 11C2 (Jongseong)
'''

CHOSEONGS = tuple([chr(i) for i in range(0x1100, 0x1113)])
TRUE_CHOSEONGS = tuple([chr(i) for i in range(0x1100, 0x1113) if chr(i) != 'ᄋ'])
JUNGSEONGS = tuple([chr(i) for i in range(0x1161, 0x1176)])
JONGSEONGS = tuple([chr(i) for i in range(0x11A8, 0x11C3)])
PUNCTUATIONS = tuple(['.', ',', '!', '?', ' '])

VALID_GRAPHEMES = CHOSEONGS + JUNGSEONGS + JONGSEONGS + PUNCTUATIONS

print(VALID_GRAPHEMES, len(VALID_GRAPHEMES))

'''
 제8항: 받침소리로는 ‘ㄱ, ㄴ, ㄷ, ㄹ, ㅁ, ㅂ, ㅇ’의 7개 자음만 발음한다.
 '''

JONGSEONGS_PRONOUNCEABLE = tuple(['ᆨ','ᆫ', 'ᆮ', 'ᆯ', 'ᆷ', 'ᆸ', 'ᆼ'])

VALID_PHONEMES = CHOSEONGS + JUNGSEONGS + JONGSEONGS_PRONOUNCEABLE + PUNCTUATIONS

print(VALID_PHONEMES, len(VALID_PHONEMES))


('ᄀ', 'ᄁ', 'ᄂ', 'ᄃ', 'ᄄ', 'ᄅ', 'ᄆ', 'ᄇ', 'ᄈ', 'ᄉ', 'ᄊ', 'ᄋ', 'ᄌ', 'ᄍ', 'ᄎ', 'ᄏ', 'ᄐ', 'ᄑ', 'ᄒ', 'ᅡ', 'ᅢ', 'ᅣ', 'ᅤ', 'ᅥ', 'ᅦ', 'ᅧ', 'ᅨ', 'ᅩ', 'ᅪ', 'ᅫ', 'ᅬ', 'ᅭ', 'ᅮ', 'ᅯ', 'ᅰ', 'ᅱ', 'ᅲ', 'ᅳ', 'ᅴ', 'ᅵ', 'ᆨ', 'ᆩ', 'ᆪ', 'ᆫ', 'ᆬ', 'ᆭ', 'ᆮ', 'ᆯ', 'ᆰ', 'ᆱ', 'ᆲ', 'ᆳ', 'ᆴ', 'ᆵ', 'ᆶ', 'ᆷ', 'ᆸ', 'ᆹ', 'ᆺ', 'ᆻ', 'ᆼ', 'ᆽ', 'ᆾ', 'ᆿ', 'ᇀ', 'ᇁ', 'ᇂ', '.', ',', '!', '?', ' ') 72
('ᄀ', 'ᄁ', 'ᄂ', 'ᄃ', 'ᄄ', 'ᄅ', 'ᄆ', 'ᄇ', 'ᄈ', 'ᄉ', 'ᄊ', 'ᄋ', 'ᄌ', 'ᄍ', 'ᄎ', 'ᄏ', 'ᄐ', 'ᄑ', 'ᄒ', 'ᅡ', 'ᅢ', 'ᅣ', 'ᅤ', 'ᅥ', 'ᅦ', 'ᅧ', 'ᅨ', 'ᅩ', 'ᅪ', 'ᅫ', 'ᅬ', 'ᅭ', 'ᅮ', 'ᅯ', 'ᅰ', 'ᅱ', 'ᅲ', 'ᅳ', 'ᅴ', 'ᅵ', 'ᆨ', 'ᆫ', 'ᆮ', 'ᆯ', 'ᆷ', 'ᆸ', 'ᆼ', '.', ',', '!', '?', ' ') 52


In [440]:
pronounce_file = glob('*.csv')[0]

with open(pronounce_file, 'r') as file:
    csv_reader = csv.reader(file)
    files = next(csv_reader)
    text = next(csv_reader)
    jamo = next(csv_reader)
    pronounce = [jamotools.split_syllables(line, 'JAMO') for line in next(csv_reader)]


In [4]:
def sanity_check(text, valid_list):
    
    result = True
    
    for char in text:
        if char not in valid_list:
            print(f'{char}, {ord(char)} in {text} is invalid')
            result = False

    return result

In [5]:
def join_jamo(jamo_text):
    
    text = jamotools.normalize_to_compat_jamo(jamo_text)
    
    return text

In [6]:
def split_jamo(text):

    jamo_text = jamotools.split_syllables(text, 'JAMO')
    
    return jamo_text

In [241]:
def get_answer_from_samples(text):
    
    text = text.strip()

    text = text.replace('ː', '')
    text = text.replace('-', '')
    text = text.replace('\n', '')

    input_output_pairs = [portion.split('[') for portion in text.split(']') if len(portion.split('[')) > 1]
    
    input_text_list = []
    output_text_list = []
    
    for pair in input_output_pairs:
        input_text_list.append(jamotools.split_syllables(pair[0], jamo_type="JAMO"))
        output_text_list.append(jamotools.split_syllables(pair[1], jamo_type="JAMO"))
    
    return input_text_list, output_text_list


In [468]:
def test_rule(rule, inputs, answers):
    
    rule_outputs = list()
    
    for sample, answer in zip(inputs, answers):
        sample = jamotools.split_syllables(sample, 'JAMO')
        rule_output = rule(sample)
        rule_outputs.append(rule_output)
        
        print(f'{sample:>10} => {rule_output:<10}: {answer:<10}')
        
    return rule_outputs

In [469]:
text = """
계집[계집]계시다[계시다]시계[시계]연계[연계]
몌별[몌별]개폐[개폐]혜택[혜택]지혜[지혜]
늴리리[닐리리]닁큼[닝큼]무늬[무니]띄어쓰기[띠어쓰기]씌어[씨어]틔어[티어]희어[히어]희떱다[히떱다]희망[히망]유희[유히]
주의[주이]협의[혀비]우리의[우리에]강의의[강이이]

"""

input_list, output_list = get_answer_from_samples(text)

print(input_list)

print(output_list)

['계집', '계시다', '시계', '연계', '몌별', '개폐', '혜택', '지혜', '늴리리', '닁큼', '무늬', '띄어쓰기', '씌어', '틔어', '희어', '희떱다', '희망', '유희', '주의', '협의', '우리의', '강의의']
['계집', '계시다', '시계', '연계', '몌별', '개폐', '혜택', '지혜', '닐리리', '닝큼', '무니', '띠어쓰기', '씨어', '티어', '히어', '히떱다', '히망', '유히', '주이', '혀비', '우리에', '강이이']


In [1129]:
split_jamo('집을 수')



'집을 수'

In [1130]:

rule_0_inputs = ['용돈을', '해안가', 
                 '있을게', '안고', '한 여자', '사람의', 
                 '거야', '바꿀 수', '십 세기',
                 '일일사']

rule_0_answers = ['용또늘', '해안까',
                '이쓸께', '안꼬', '한 녀자', '사라메', 
                '꺼야', '바꿀 쑤', '십 쎄기',
                '#ᄋ#ᅵ#ᆯ#ᄋ#ᅵ#ᆯ#ᄉ#ᅡ#']

def rule_0(input_text, verbose=False):
        
    graphenes = ['용돈을', '해안가', 
                 '있을게', '안고', '한 여자', '사람의', 
                 '거야', '바꿀 수', '십 세기',
                 '색연필', '밝힐 수는', '조건',
                 '발생', '남의', '만 원을', '벗을 수가',
                 '그의', '게요', '누군가의', 
                 '절대', '안지', '실수', '몇 장',
                 '수가', '할아버지의', '다섯 병',
                 '열쇠', '그들의', '몇 시', '차의', '홉 살에', 
                 '환자의', '수업 중', '마음의',
                 '그녀의', '부모님의',
                 '사당역', '일기장', '나을 것', '일곱 시', '회사의', '걸릴 거', 
                 '좋을 것', '한국의', '할 거', '할 곳', '일자리', '사건', '결정',
                 '결심', '집을 수', '십 년', '한밤중', '깎을 줄',
                 '않을 거', '여권', '어릴 적', '잠잘 시간', '물질',
                 '먹을 것', '낼 건', '', '',
                 '일일사']
    phonemes = ['용또늘', '해안까',
                '이쓸께', '안꼬', '한 녀자', '사라메', 
                '꺼야', '바꿀 쑤', '십 쎄기',
                '생년필', '발킬 쑤는', '조껀',
                '발쌩', '나메', '마 눠늘', '버슬 쑤가',
                '그에', '께요', '누군가에',
                '절때', '안찌', '실쑤', '멷 짱',
                '쑤가', '하라버지에', '다섣 뼝',
                '열쐬', '그들에', '몇 씨', '차에', '홉 싸레', 
                '환자에', '수업 쭝', '마으메',
                '그녀에', '부모니메',
                '사당녁', '일기짱', '나을 껃', '일곱 씨', '회사에', '걸릴 꺼', 
                '조을 껃', '한구게', '할 꺼', '할 꼿', '일짜리', '사껀', '결쩡',
                '결씸', '지블 쑤', '심 년', '한밤쭝', '까끌 쭐', 
                '아늘 꺼', '여꿘', '어릴 쩍', '잠잘 씨간', '물찔', 
                '머글 껏', '낼 껀', '', '',
                '#ᄋ#ᅵ#ᆯ#ᄋ#ᅵ#ᆯ#ᄉ#ᅡ#']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
    
    return input_text

_ = test_rule(rule_0, rule_0_inputs, rule_0_answers)

 용돈을 => 용또늘  : 용또늘  
   해안가 => 해안까   : 해안까   
  있을게 => 이쓸께   : 이쓸께   
     안고 => 안꼬     : 안꼬     
  한 여자 => 한 녀자  : 한 녀자  
   사람의 => 사라메    : 사라메    
      거야 => 꺼야      : 꺼야      
  바꿀 수 => 바꿀 쑤  : 바꿀 쑤  
  십 세기 => 십 쎄기  : 십 쎄기  
  일일사 => #ᄋ#ᅵ#ᆯ#ᄋ#ᅵ#ᆯ#ᄉ#ᅡ#: #ᄋ#ᅵ#ᆯ#ᄋ#ᅵ#ᆯ#ᄉ#ᅡ#


In [1131]:
def rule_last(input_text, verbose=False):
        
    input_text = input_text.replace('#', '')
    
    return input_text

In [1132]:
# 제5항 ‘ㅑ ㅒ ㅕ ㅖ ㅘ ㅙ ㅛ ㅝ ㅞ ㅠ ㅢ’는 이중 모음으로 발음한다.
# 다만 1. 용언의 활용형에 나타나는 ‘져, 쪄, 쳐’는 [저, 쩌, 처]로 발음한다. 
# 다만 2. ‘예, 례' 이외의 ‘ㅖ’는 [ㅔ]로도 발음한다. 
# 다만 3. 자음을 첫소리로 가지고 있는 음절의 ‘ㅢ’는 [ㅣ]로 발음한다. 
# 다만 4. 단어의 첫음절 이외의 ‘의’는 [ㅣ]로, 조사 ‘의’는 [ㅔ]로 발음함도 허용한다. 

rule_5_inputs = ['가져', '쪄', '다쳐',
                 '계집', '계시다', '시계', '연계', '몌별', 
                 '개폐', '혜택', '지혜', '늴리리', '닁큼', 
                 '무늬', '띄어쓰기', '씌어', '틔어', '희어', 
                 '희떱다', '희망', '유희', '주의', '협의', '우리의', '강의의']

rule_5_answers = ['가저', '쩌', '다처',
                  '계집', '계시다', '시계', '연계', '몌별', 
                  '개폐', '혜택', '지혜', '닐리리', '닝큼', 
                  '무니', '띠어쓰기', '씨어', '티어', '히어', 
                  '히떱다', '히망', '유히', '주이', '혀비', '우리에', '강이에']

def rule_5(input_text, verbose=False):
    
    input_text = input_text.replace('져', '저')
    input_text = input_text.replace('쪄', '쩌')
    input_text = input_text.replace('쳐', '처')
    
    text_as_list = list(input_text)
    
    target_j_groups = (('ᆩ', 'ᆿ'), ('ᆺ', 'ᆻ', 'ᆽ', 'ᆾ', 'ᇀ'), ('ᇁ'))
    pronounciation_group = ('ᆨ', 'ᆮ', 'ᆸ')
    
    for i, char in enumerate(text_as_list):
#         if char == 'ᅨ' and text_as_list[i-1] in CHOSEONGS and text_as_list[i-1] not in ('ᄋ', 'ᄅ'):
#             text_as_list[i] = 'ᅦ'
            
        if char == 'ᅴ' and text_as_list[i-1] in CHOSEONGS and text_as_list[i-1] != 'ᄋ':
            text_as_list[i] = 'ᅵ'
        
        try:
            if char == 'ᅴ' and text_as_list[i-1] == 'ᄋ' and text_as_list[i-2] != ' ':
                text_as_list[i] = 'ᅵ'
        except IndexError:
            pass
        
    return ''.join(text_as_list)

_ = test_rule(rule_5, rule_5_inputs, rule_5_answers)


      가져 => 가저      : 가저      
        쪄 => 쩌        : 쩌        
      다쳐 => 다처      : 다처      
     계집 => 계집     : 계집     
    계시다 => 계시다    : 계시다    
      시계 => 시계      : 시계      
     연계 => 연계     : 연계     
     몌별 => 몌별     : 몌별     
      개폐 => 개폐      : 개폐      
     혜택 => 혜택     : 혜택     
      지혜 => 지혜      : 지혜      
   늴리리 => 닐리리   : 닐리리   
    닁큼 => 닝큼    : 닝큼    
      무늬 => 무니      : 무니      
  띄어쓰기 => 띠어쓰기  : 띠어쓰기  
      씌어 => 씨어      : 씨어      
      틔어 => 티어      : 티어      
      희어 => 히어      : 히어      
   희떱다 => 히떱다   : 히떱다   
     희망 => 히망     : 히망     
      유희 => 유히      : 유히      
      주의 => 주이      : 주이      
     협의 => 협이     : 혀비      
    우리의 => 우리이    : 우리에    
   강의의 => 강이이   : 강이에    


In [1133]:
# 제9항 받침 ‘ㄲ, ㅋ’, ‘ㅅ, ㅆ, ㅈ, ㅊ, ㅌ’, ‘ㅍ’은 어말 또는 자음 앞에서 각각 대표음 [ㄱ, ㄷ, ㅂ]으로 발음한다.

rule_9_inputs = ['닦다', '키읔', '키읔과', '옷', '웃다', 
                  '있다', '젖', '빚다', '꽃', '쫓다', 
                  '솥', '뱉다', '앞', '덮다']

rule_9_answers = ['닥따', '키윽', '키윽꽈', '옫', '욷따', 
                  '읻따', '젇', '빋따', '꼳', '쫃따', 
                  '솓', '밷따', '압', '덥따']

def rule_9(input_text, verbose=False):
    
    text_as_list = list(input_text)
    
    target_jongseong_groups = (('ᆩ', 'ᆿ'), ('ᆺ', 'ᆻ', 'ᆽ', 'ᆾ', 'ᇀ'), ('ᇁ'))
    pronounciation_group = ('ᆨ', 'ᆮ', 'ᆸ')
    
    for i, char in enumerate(input_text):
        for j, group in enumerate(target_jongseong_groups):
            try:
                if char in group and input_text[i+1] in (PUNCTUATIONS + TRUE_CHOSEONGS) :
                    text_as_list[i] = pronounciation_group[j]

                    if verbose: print('Rule 9!')
                        
            except IndexError:
                text_as_list[i] = pronounciation_group[j]
                if verbose: print('Rule 9!')
                pass # Checking beyond the end of a sentence
    
    return ''.join(text_as_list)

_ = test_rule(rule_9, rule_9_inputs, rule_9_answers)


     닦다 => 닥다     : 닥따     
     키읔 => 키윽     : 키윽     
   키읔과 => 키윽과   : 키윽꽈   
       옷 => 옫       : 옫       
     웃다 => 욷다     : 욷따     
     있다 => 읻다     : 읻따     
       젖 => 젇       : 젇       
     빚다 => 빋다     : 빋따     
       꽃 => 꼳       : 꼳       
     쫓다 => 쫃다     : 쫃따     
       솥 => 솓       : 솓       
     뱉다 => 밷다     : 밷따     
       앞 => 압       : 압       
     덮다 => 덥다     : 덥따     


In [1134]:
# 제10항 겹받침 ‘ㄳ’, ‘ㄵ’, ‘ㄼ, ㄽ, ㄾ’, ‘ㅄ’은 어말 또는 자음 앞에서 각각 [ㄱ, ㄴ, ㄹ, ㅂ]으로 발음한다.
# 다만, ‘밟-’은 자음 앞에서 [밥]으로 발음하고, ‘넓-’은 다음과 같은 경우에 [넙]으로 발음한다.
# (2) 넓-죽하다[넙쭈카다]넓-둥글다[넙뚱글다]

rule_10_inputs = ['넋', '넋과', '앉다', '여덟', '넓다', '외곬', '핥다', '값', '없다']
rule_10_answers = ['넉', '넉꽈', '안따', '여덜', '널따', '외골', '할따', '갑', '업따']

def rule_10(input_text, verbose=False):
    
    text_as_list = list(input_text)
    
    target_jongseong_groups = (('ᆪ'), ('ᆬ'), ('ᆲ', 'ᆳ', 'ᆴ'), ('ᆹ'))
    pronounciation_group = ('ᆨ', 'ᆫ', 'ᆯ', 'ᆸ')
    
    for i, char in enumerate(input_text):
        for j, group in enumerate(target_jongseong_groups):
            try:
                if char in group and input_text[i+1] in (PUNCTUATIONS + TRUE_CHOSEONGS) :
                    text_as_list[i] = pronounciation_group[j]
                    if verbose: print('Rule 10!')
            except IndexError:
                text_as_list[i] = pronounciation_group[j]
                if verbose: print('Rule 10!')
                pass
                
    
    return ''.join(text_as_list)


_ = test_rule(rule_10, rule_10_inputs, rule_10_answers)

       넋 => 넉       : 넉       
     넋과 => 넉과     : 넉꽈     
     앉다 => 안다     : 안따     
     여덟 => 여덜     : 여덜     
     넓다 => 널다     : 널따     
     외곬 => 외골     : 외골     
     핥다 => 할다     : 할따     
       값 => 갑       : 갑       
     없다 => 업다     : 업따     


In [1135]:
# 제11항 겹받침 ‘ㄺ, ㄻ, ㄿ’은 어말 또는 자음 앞에서 각각 [ㄱ, ㅁ, ㅂ]으로 발음한다.
# 다만, 용언의 어간 말음 ‘ㄺ’은 ‘ㄱ’ 앞에서 [ㄹ]로 발음한다. 

rule_11_inputs = ['닭', '흙과', '맑다', '늙지', '삶', 
                  '젊다', '읊고', '읊다', '맑게', '묽고', '얽거나']

rule_11_answers = ['닥', '흑꽈', '막따', '늑찌', '삼', 
                   '점따', '읍꼬', '읍따', '말께', '물꼬', '얼꺼나']

def rule_11(input_text, verbose=False):
    
    text_as_list = list(input_text)
    
    target_jongseong_groups = (('ᆰ'), ('ᆱ'), ('ᆵ'))
    pronounciation_group = ('ᆨ', 'ᆷ', 'ᆸ')
    
    for i, char in enumerate(text_as_list):
        try:
            if char == 'ᆰ' and input_text[i+1] == 'ᄀ':
                text_as_list[i] = 'ᆯ'
                if verbose: print('Rule 11 Special!')
        except IndexError:
            pass
            
    
    for i, char in enumerate(text_as_list):
        for j, group in enumerate(target_jongseong_groups):
            try:
                if char in group and input_text[i+1] in (PUNCTUATIONS + TRUE_CHOSEONGS) :
                    text_as_list[i] = pronounciation_group[j]

                    if verbose: print('Rule 11!')
                        
            except IndexError:
                text_as_list[i] = pronounciation_group[j]
                if verbose: print('Rule 11!')
                pass
                
    
    return ''.join(text_as_list)

_ = test_rule(rule_11, rule_11_inputs, rule_11_answers)

       닭 => 닥       : 닥       
     흙과 => 흘과     : 흑꽈     
     맑다 => 막다     : 막따     
     늙지 => 늑지     : 늑찌     
       삶 => 삼       : 삼       
     젊다 => 점다     : 점따     
     읊고 => 읍고     : 읍꼬     
     읊다 => 읍다     : 읍따     
     맑게 => 말게     : 말께     
     묽고 => 물고     : 물꼬     
   얽거나 => 얼거나   : 얼꺼나   


In [1136]:
# 제12항 받침 ‘ㅎ’의 발음은 다음과 같다.
# 1. ‘ㅎ(ㄶ, ㅀ)’ 뒤에 ‘ㄱ, ㄷ, ㅈ’이 결합되는 경우에는, 뒤 음절 첫소리와 합쳐서 [ㅋ, ㅌ, ㅊ]으로 발음한다. 
# [붙임 1] 받침 ‘ㄱ(ㄺ), ㄷ, ㅂ(ㄼ), ㅈ(ㄵ)’이 뒤 음절 첫소리 ‘ㅎ’과 결합되는 경우에도, 역시 두 음을 합쳐서 [ㅋ, ㅌ, ㅍ, ㅊ]으로 발음한다.
# [붙임 2] 규정에 따라 'ㄷ'으로 발음되는 ‘ㅅ, ㅈ, ㅊ, ㅌ’의 경우에도 이에 준한다. 
# 2. ‘ㅎ(ㄶ, ㅀ)’ 뒤에 ‘ㅅ’이 결합되는 경우에는, ‘ㅅ’을 [ㅆ]으로 발음한다. 
# 3. ‘ㅎ’ 뒤에 ‘ㄴ’이 결합되는 경우에는, [ㄴ]으로 발음한다.
# [붙임] ‘ㄶ, ㅀ’ 뒤에 ‘ㄴ’이 결합되는 경우에는, ‘ㅎ’을 발음하지 않는다. 
# 4. ‘ㅎ(ㄶ, ㅀ)’ 뒤에 모음으로 시작된 어미나 접미사가 결합되는 경우에는, ‘ㅎ’을 발음하지 않는다. 

rule_12_inputs = ['놓고', '좋던', '쌓지', '많고', '않던', 
                  '닳지', '각하', '먹히다', '밝히다', '맏형', 
                  '좁히다', '넓히다', '꽂히다', '앉히다', 
                  '옷 한 벌', '낮 한때', '꽃 한 송이', '숱하다', 
                  '닿소', '많소', '싫소', '놓는', '쌓네', '않네', 
                  '않는', '뚫네', '뚫는', '낳은', '놓아', '쌓이다', 
                  '많아', '않은', '닳아', '싫어도']

rule_12_answers = ['노코', '조턴', '싸치', '만코', '안턴', 
                   '달치', '가카', '머키다', '발키다', '마텽', 
                   '조피다', '널피다', '꼬치다', '안치다', 
                   '오탄벌', '나탄때', '꼬탄송이', '수타다', 
                   '다쏘', '만쏘', '실쏘', '논는', '싼네', '안네', 
                   '안는', '뚤네→뚤레', '뚤는→뚤른', '나은', '노아', '싸이다', 
                   '마나', '아는', '다라', '시러도']

def rule_12(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    pop_assignment_list = list()
    
    for i, char in enumerate(text_as_list):

        if char in ('ᇂ', 'ᆭ', 'ᆶ'): 
            if text_as_list[i+1] == 'ᄀ': text_as_list[i+1] = 'ᄏ'
            elif text_as_list[i+1] == 'ᄃ': text_as_list[i+1] = 'ᄐ'
            elif text_as_list[i+1] == 'ᄌ': text_as_list[i+1] = 'ᄎ'
            elif text_as_list[i+1] == 'ᄉ': text_as_list[i+1] = 'ᄊ'
            
            if text_as_list[i] == 'ᇂ': 
                if text_as_list[i+1] == 'ᄂ':
                    text_as_list[i] = 'ᆫ'
                else:   
                    pop_assignment_list.append(i)
            elif text_as_list[i] == 'ᆭ': text_as_list[i] = 'ᆫ'
            elif text_as_list[i] == 'ᆶ': text_as_list[i] = 'ᆯ'
                
        if char in ('ᆨ', 'ᆰ', 'ᆮ', 'ᆸ', 'ᆲ', 'ᆽ', 'ᆬ') and text_as_list[i+1] == 'ᄒ': 
            if char in ('ᆨ', 'ᆰ'): 
                text_as_list[i+1] = 'ᄏ'
                if char == 'ᆨ':
                    pop_assignment_list.append(i)
                elif char == 'ᆰ':
                    text_as_list[i] = 'ᆯ'
            elif char in ('ᆮ'): 
                text_as_list[i+1] = 'ᄐ'
                pop_assignment_list.append(i)
            elif char in ('ᆸ', 'ᆲ'): 
                text_as_list[i+1] = 'ᄑ'
                if char == 'ᆸ':
                    pop_assignment_list.append(i)
                elif char == 'ᆲ':
                    text_as_list[i] = 'ᆯ'
            elif char in ('ᆽ', 'ᆬ'): 
                text_as_list[i+1] = 'ᄎ'
                if char == 'ᆽ':
                    pop_assignment_list.append(i)
                elif char == 'ᆬ':
                    text_as_list[i] = 'ᆫ'
                
#             elif text_as_list[i+1] == 'ᄌ': text_as_list[i+1] = 'ᄎ'
            
#             if text_as_list[i] == 'ᇂ': pop_assignment_list.append(i)
#             elif text_as_list[i] == 'ᆭ': text_as_list[i] = 'ᆫ'
#             elif text_as_list[i] == 'ᆶ': text_as_list[i] = 'ᆯ'
            
            if verbose: print('Rule 12 Special!')
    
    for pop_index in sorted(pop_assignment_list, reverse=True):
        text_as_list.pop(pop_index)
    
    return ''.join(text_as_list).strip()

_ = test_rule(rule_12, rule_12_inputs, rule_12_answers)
# _ = test_rule(rule_13, _, rule_12_answers)

     놓고 => 노코      : 노코      
    좋던 => 조턴     : 조턴     
     쌓지 => 싸치      : 싸치      
     많고 => 만코     : 만코     
    않던 => 안턴    : 안턴    
     닳지 => 달치     : 달치     
     각하 => 가카      : 가카      
   먹히다 => 머키다    : 머키다    
   밝히다 => 발키다   : 발키다   
    맏형 => 마텽     : 마텽     
   좁히다 => 조피다    : 조피다    
   넓히다 => 널피다   : 널피다   
   꽂히다 => 꼬치다    : 꼬치다    
   앉히다 => 안치다   : 안치다   
옷 한 벌 => 옷 한 벌: 오탄벌  
 낮 한때 => 낮 한때 : 나탄때   
꽃 한 송이 => 꽃 한 송이: 꼬탄송이
   숱하다 => 숱하다   : 수타다    
     닿소 => 다쏘      : 다쏘      
     많소 => 만쏘     : 만쏘     
     싫소 => 실쏘     : 실쏘     
    놓는 => 논는    : 논는    
     쌓네 => 싼네     : 싼네     
     않네 => 안네     : 안네     
    않는 => 안는    : 안는    
     뚫네 => 뚤네     : 뚤네→뚤레
    뚫는 => 뚤는    : 뚤

In [1137]:
# 제13항 홑받침이나 쌍받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 제 음가대로 뒤 음절 첫소리로 옮겨 발음한다.

rule_13_inputs = ['깎아', '옷이', '있어', '낮이', '꽂아', '꽃을', '쫓아', '밭에', '앞으로', '덮이다']

rule_13_answers = ['까까', '오시', '이써', '나지', '꼬자', '꼬츨', '쪼차', '바테', '아프로', '더피다']

# ㅇ 은 받침이 아니다!

def rule_13(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    target_jongseong_groups = ('ᆨ', 'ᆩ', 'ᆫ', 'ᆮ', 'ᆯ', 
                               'ᆷ', 'ᆸ', 'ᆺ', 'ᆻ', # 'ᆼ', 
                               'ᆽ', 'ᆾ', 'ᆿ', 'ᇀ', 'ᇁ', 'ᇂ')
    pronounciation_group = ('ᄀ', 'ᄁ', 'ᄂ', 'ᄃ', 'ᄅ', 
                            'ᄆ', 'ᄇ', 'ᄉ', 'ᄊ', # 'ᄋ', 
                            'ᄌ', 'ᄎ', 'ᄏ', 'ᄐ', 'ᄑ', 'ᄒ')
    
    pop_assignment_list = list()
    
    for i, char in enumerate(text_as_list):
        if char in target_jongseong_groups and text_as_list[i+1] == 'ᄋ':
            text_as_list[i+1] = pronounciation_group[target_jongseong_groups.index(char)]
            pop_assignment_list.append(i)
            
    for pop_index in sorted(pop_assignment_list, reverse=True):
        text_as_list.pop(pop_index)
                
    
    return ''.join(text_as_list).strip()

_ = test_rule(rule_13, rule_13_inputs, rule_13_answers)


     깎아 => 까까      : 까까      
     옷이 => 오시      : 오시      
     있어 => 이써      : 이써      
     낮이 => 나지      : 나지      
     꽂아 => 꼬자      : 꼬자      
    꽃을 => 꼬츨     : 꼬츨     
     쫓아 => 쪼차      : 쪼차      
     밭에 => 바테      : 바테      
   앞으로 => 아프로    : 아프로    
   덮이다 => 더피다    : 더피다    


In [1138]:
# 제14항 겹받침이 모음으로 시작된 조사나 어미, 접미사와 결합되는 경우에는, 
# 뒤엣것만을 뒤 음절 첫소리로 옮겨 발음한다.(이 경우, ‘ㅅ’은 된소리로 발음함.)

rule_14_inputs = ['넋이', '앉아', '닭을', '젊어', '곬이', '핥아', '읊어', '값을', '없어', '닭이', '닭을', '여덟이', '여덟을']

rule_14_answers = ['넉씨', '안자', '달글', '절머', '골씨', '할타', '을퍼', '갑쓸', '업써', '달기', '달글', '여덜비', '여덜블']


def rule_14(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    target_jongseong_groups = ('ᆪ', 'ᆬ', 'ᆭ', 'ᆰ', 
                               'ᆱ', 'ᆲ', 'ᆳ', 'ᆴ', 'ᆵ', 
                               'ᆶ', 'ᆹ')
    
    pronounciation_group = ('ᄊ', 'ᄌ', 'ᄒ', 'ᄀ',
                            'ᄆ', 'ᄇ', 'ᄊ', 'ᄐ', 'ᄑ',
                            'ᄒ', 'ᄊ')
    
    replacement_group = ('ᆨ', 'ᆫ', 'ᆫ', 'ᆯ',
                         'ᆯ', 'ᆯ', 'ᆯ', 'ᆯ', 'ᆯ',
                         'ᆯ', 'ᆸ')
    
    for i, char in enumerate(text_as_list):
        if char in target_jongseong_groups and text_as_list[i+1] == 'ᄋ':
            text_as_list[i+1] = pronounciation_group[target_jongseong_groups.index(char)]
            text_as_list[i] = replacement_group[target_jongseong_groups.index(char)]    
    
    return ''.join(text_as_list).strip()

_ = test_rule(rule_14, rule_14_inputs, rule_14_answers)

     넋이 => 넉씨     : 넉씨     
     앉아 => 안자     : 안자     
    닭을 => 달글    : 달글    
     젊어 => 절머     : 절머     
     곬이 => 골씨     : 골씨     
     핥아 => 할타     : 할타     
     읊어 => 을퍼     : 을퍼     
    값을 => 갑쓸    : 갑쓸    
     없어 => 업써     : 업써     
     닭이 => 달기     : 달기     
    닭을 => 달글    : 달글    
   여덟이 => 여덜비   : 여덜비   
  여덟을 => 여덜블  : 여덜블  


In [1139]:
# 제15항 받침 뒤에 모음 ‘ㅏ, ㅓ, ㅗ, ㅜ, ㅟ’ 들로 시작되는 실질 형태소가 연결되는 경우에는, 
# 대표음으로 바꾸어서 뒤 음절 첫소리로 옮겨 발음한다.
# [붙임] 겹받침의 경우에는, 그중 하나만을 옮겨 발음한다. 

rule_15_inputs = ['밭 아래', '늪 앞', '젖어미', '맛없다', '겉옷', '헛웃음', '꽃 위', '넋 없다', '닭 앞에', '값어치', '값있는']

rule_15_answers = ['바다래', '느밥', '저더미', '마덥따', '거돋', '허두슴', '꼬뒤', '너겁따', '다가페', '가버치', '가빈는']


def rule_15(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    target_jongseong_groups = ('ᆨ', 'ᆪ', 'ᆫ', 'ᆬ', 
                               'ᆭ', 'ᆮ', 'ᆯ', 'ᆰ', 'ᆱ', 
                               'ᆲ', 'ᆳ', 'ᆴ', 'ᆵ', 'ᆶ', 
                               'ᆷ', 'ᆸ', 'ᆹ', 'ᆺ', 
                               # 'ᆼ', 
                               'ᆽ', 'ᆾ', 'ᆿ', 'ᇀ', 
                               'ᇁ', 'ᇂ')
    
    pronounciation_group = ('ᄀ', 'ᄀ', 'ᄂ', 'ᄌ', 
                            'ᄂ', 'ᄃ', 'ᄅ', 'ᄀ', 'ᄆ',
                            'ᄇ', 'ᄊ', 'ᄐ', 'ᄑ', 'ᄅ',
                            'ᄆ', 'ᄇ', 'ᄇ', 'ᄃ', 
                            # 'ᄋ', 
                            'ᄃ', 'ᄃ', 'ᄀ', 'ᄃ',
                            'ᄇ', 'ᄒ')
    
    jungseongs = ('ᅡ', 'ᅥ', 'ᅩ', 'ᅮ', 'ᅱ')
    
    pop_assignment_list = list()
    
    for i, char in enumerate(text_as_list):
        if char in target_jongseong_groups and text_as_list[i+1] == 'ᄋ' and text_as_list[i+2] in jungseongs:
            text_as_list[i+1] = pronounciation_group[target_jongseong_groups.index(char)]
            pop_assignment_list.append(i)
    
    for pop_index in sorted(pop_assignment_list, reverse=True):
        text_as_list.pop(pop_index)
            
    
    return ''.join(text_as_list).strip()

rule_15_inputs = [input_.replace(' ', '') for input_ in rule_15_inputs]

_ = test_rule(rule_15, rule_15_inputs, rule_15_answers)
# _ = test_rule(rule_14, _, rule_15_answers)

   밭아래 => 바다래    : 바다래    
    늪앞 => 느밮     : 느밥     
   젖어미 => 저더미    : 저더미    
  맛없다 => 마덦다   : 마덥따   
    겉옷 => 거돗     : 거돋     
 헛웃음 => 허둣음  : 허두슴   
     꽃위 => 꼬뒤      : 꼬뒤      
  넋없다 => 너겂다   : 너겁따   
  닭앞에 => 다갚에   : 다가페    
   값어치 => 가버치    : 가버치    
 값있는 => 값있는 : 가빈는  


In [1140]:
# 제16항 한글 자모의 이름은 그 받침소리를 연음하되, ‘ㄷ, ㅈ, ㅊ, ㅋ, ㅌ, ㅍ, ㅎ’의 경우에는 특별히 다음과 같이 발음한다.

rule_16_inputs = ['디귿이', '디귿을', '디귿에', 
                  '지읒이', '지읒을', '지읒에', 
                  '치읓이', '치읓을', '치읓에', 
                  '키읔이', '키읔을', '키읔에', 
                  '티읕이', '티읕을', '티읕에', 
                  '피읖이', '피읖을', '피읖에', 
                  '히읗이', '히읗을', '히읗에']

rule_16_answers = ['디그시', '디그슬', '디그세', 
                   '지으시', '지으슬', '지으세', 
                   '치으시', '치으슬', '치으세', 
                   '키으기', '키으글', '키으게', 
                   '티으시', '티으슬', '티으세', 
                   '피으비', '피으블', '피으베', 
                   '히으시', '히으슬', '히으세']

def rule_16(input_text, verbose=False):
        
    graphenes = ['디귿이', '디귿을', '디귿에', 
                  '지읒이', '지읒을', '지읒에', 
                  '치읓이', '치읓을', '치읓에', 
                  '키읔이', '키읔을', '키읔에', 
                  '티읕이', '티읕을', '티읕에', 
                  '피읖이', '피읖을', '피읖에', 
                  '히읗이', '히읗을', '히읗에']
    
    phonemes = ['디그시', '디그슬', '디그세', 
                   '지으시', '지으슬', '지으세', 
                   '치으시', '치으슬', '치으세', 
                   '키으기', '키으글', '키으게', 
                   '티으시', '티으슬', '티으세', 
                   '피으비', '피으블', '피으베', 
                   '히으시', '히으슬', '히으세']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
    
    return input_text

_ = test_rule(rule_16, rule_16_inputs, rule_16_answers)


   디귿이 => 디그시    : 디그시    
  디귿을 => 디그슬   : 디그슬   
   디귿에 => 디그세    : 디그세    
   지읒이 => 지으시    : 지으시    
  지읒을 => 지으슬   : 지으슬   
   지읒에 => 지으세    : 지으세    
   치읓이 => 치으시    : 치으시    
  치읓을 => 치으슬   : 치으슬   
   치읓에 => 치으세    : 치으세    
   키읔이 => 키으기    : 키으기    
  키읔을 => 키으글   : 키으글   
   키읔에 => 키으게    : 키으게    
   티읕이 => 티으시    : 티으시    
  티읕을 => 티으슬   : 티으슬   
   티읕에 => 티으세    : 티으세    
   피읖이 => 피으비    : 피으비    
  피읖을 => 피으블   : 피으블   
   피읖에 => 피으베    : 피으베    
   히읗이 => 히으시    : 히으시    
  히읗을 => 히으슬   : 히으슬   
   히읗에 => 히으세    : 히으세    


In [1141]:
# 제17항 받침 ‘ㄷ, ㅌ(ㄾ)’이 조사나 접미사의 모음 ‘ㅣ’와 결합되는 경우에는, [ㅈ, ㅊ]으로 바꾸어서 뒤 음절 첫소리로 옮겨 발음한다.
# [붙임] ‘ㄷ’ 뒤에 접미사 ‘히’가 결합되어 ‘티’를 이루는 것은 [치]로 발음한다. 

rule_17_inputs = ['곧이듣다', '굳이', '미닫이', '땀받이', '밭이', '벼훑이', '굳히다', '닫히다', '묻히다']

rule_17_answers = ['고지듣따', '구지', '미다지', '땀바지', '바치', '벼훌치', '구치다', '다치다', '무치다']


def rule_17(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    target_jongseong_groups = ('ᆮ', 'ᆴ', 'ᇀ')
    
    pronounciation_group = ('ᄌ', 'ᄎ', 'ᄎ')
    
    pop_assignment_list = list()
    
    for i, char in enumerate(text_as_list):
        if char in target_jongseong_groups and text_as_list[i+1] == 'ᄋ' and text_as_list[i+2] == 'ᅵ':
            text_as_list[i+1] = pronounciation_group[target_jongseong_groups.index(char)]
            if char in ('ᆮ','ᇀ'):
                pop_assignment_list.append(i)
            elif char == 'ᆴ':
                text_as_list[i] = 'ᆯ'
        elif char == 'ᆮ' and text_as_list[i+1] == 'ᄒ' and text_as_list[i+2] == 'ᅵ':
            text_as_list[i+1] = 'ᄎ'
            pop_assignment_list.append(i)
    
    for pop_index in sorted(pop_assignment_list, reverse=True):
        text_as_list.pop(pop_index)
    
    return ''.join(text_as_list).strip()

_ = test_rule(rule_17, rule_17_inputs, rule_17_answers)


곧이듣다 => 고지듣다 : 고지듣따 
     굳이 => 구지      : 구지      
   미닫이 => 미다지    : 미다지    
  땀받이 => 땀바지   : 땀바지   
     밭이 => 바치      : 바치      
   벼훑이 => 벼훌치   : 벼훌치   
   굳히다 => 구치다    : 구치다    
   닫히다 => 다치다    : 다치다    
   묻히다 => 무치다    : 무치다    


In [1142]:
# 제18항 받침 ‘ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ, ㅎ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)’은 ‘ㄴ, ㅁ’ 앞에서 [ㅇ, ㄴ, ㅁ]으로 발음한다.

rule_18_inputs = ['먹는', '국물', '깎는', '키읔만', '몫몫이', 
                  '긁는', '흙만', '닫는', '짓는', '옷맵시', 
                  '있는', '맞는', '젖멍울', '쫓는', '꽃망울', 
                  '붙는', '놓는', '잡는', '밥물', '앞마당', 
                  '밟는', '읊는', '없는', '책 넣는다', '흙 말리다', 
                  '옷 맞추다', '밥 먹는다', '값 매기다']

rule_18_answers = ['멍는', '궁물', '깡는', '키응만', '몽목씨', 
                   '긍는', '흥만', '단는', '진는', '온맵씨', 
                   '인는', '만는', '전멍울', '쫀는', '꼰망울', 
                   '분는', '논는', '잠는', '밤물', '암마당', 
                   '밤는', '음는', '엄는', '챙넌는다', '흥말리다', 
                   '온맏추다', '밤멍는다', '감매기다']


def rule_18(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    target_jongseong_groups = (('ᆨ', 'ᆩ', 'ᆿ', 'ᆪ', 'ᆰ'),
                               ('ᆮ', 'ᆺ', 'ᆻ', 'ᆽ', 'ᆾ', 'ᇀ', 'ᇂ'),
                               ('ᆸ', 'ᇁ','ᆲ', 'ᆵ','ᆹ'))
    
    pronounciation_group = ('ᆼ', 'ᆫ', 'ᆷ')
    
    for i, char in enumerate(text_as_list):
        for j, group in enumerate(target_jongseong_groups):
            if char in group and text_as_list[i+1] in ('ᄂ', 'ᄆ'):
                text_as_list[i] = pronounciation_group[j]
    
    return ''.join(text_as_list).strip()

rule_18_inputs = [input_.replace(' ', '') for input_ in rule_18_inputs]

_ = test_rule(rule_18, rule_18_inputs, rule_18_answers)

    먹는 => 멍는    : 멍는    
    국물 => 궁물    : 궁물    
    깎는 => 깡는    : 깡는    
  키읔만 => 키응만  : 키응만  
  몫몫이 => 몽몫이  : 몽목씨  
    긁는 => 긍는    : 긍는    
    흙만 => 흥만    : 흥만    
    닫는 => 단는    : 단는    
    짓는 => 진는    : 진는    
  옷맵시 => 온맵시  : 온맵씨  
    있는 => 인는    : 인는    
    맞는 => 만는    : 만는    
 젖멍울 => 전멍울 : 전멍울 
    쫓는 => 쫀는    : 쫀는    
 꽃망울 => 꼰망울 : 꼰망울 
    붙는 => 분는    : 분는    
    놓는 => 논는    : 논는    
    잡는 => 잠는    : 잠는    
    밥물 => 밤물    : 밤물    
  앞마당 => 암마당  : 암마당  
    밟는 => 밤는    : 밤는    
    읊는 => 음는    : 음는    
    없는 => 엄는    : 엄는    
책넣는다 => 챙넌는다: 챙넌는다
흙말리다 => 흥말리다: 흥말리다
옷맞추다 => 온맞추다: 온맏추다
밥먹는다 => 밤멍는다: 밤멍ᄂ

In [1143]:
text = """
담력[담ː녁]침략[침ː냑]강릉[강능]항로[항ː노]대통령[대ː통녕]막론[막논→망논]석류[석뉴→성뉴]협력[협녁→혐녁]법리[법니→범니]
"""

input_list, answer_list = get_answer_from_samples(text)

print(input_list)

print(answer_list)

['담력', '침략', '강릉', '항로', '대통령', '막론', '석류', '협력', '법리']
['담녁', '침냑', '강능', '항노', '대통녕', '막논→망논', '석뉴→성뉴', '협녁→혐녁', '법니→범니']


In [1144]:
# 제19항 받침 ‘ㅁ, ㅇ’ 뒤에 연결되는 ‘ㄹ’은 [ㄴ]으로 발음한다.

rule_19_inputs = ['담력', '침략', '강릉', '항로', '대통령', 
                  '막론', '석류', '협력', '법리']

rule_19_answers = ['담녁', '침냑', '강능', '항노', '대통녕', 
                   '막논→망논', '석뉴→성뉴', '협녁→혐녁', '법니→범니']

def rule_19(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    for i, char in enumerate(text_as_list):
        if char in ('ᆷ', 'ᆼ', 'ᆨ', 'ᆸ') and text_as_list[i+1] == 'ᄅ':
            text_as_list[i+1] = 'ᄂ'
            
    return ''.join(text_as_list).strip()

_ = test_rule(rule_19, rule_19_inputs, rule_19_answers)

    담력 => 담녁    : 담녁    
    침략 => 침냑    : 침냑    
    강릉 => 강능    : 강능    
     항로 => 항노     : 항노     
  대통령 => 대통녕  : 대통녕  
    막론 => 막논    : 막논→망논
     석류 => 석뉴     : 석뉴→성뉴
    협력 => 협녁    : 협녁→혐녁
     법리 => 법니     : 법니→범니


In [1145]:
text = """
난로[날ː로]신라[실라]천리[철리]광한루[광ː할루]대관령[대ː괄령]
칼날[칼랄]물난리[물랄리]줄넘기[줄럼끼]할는지[할른지]
닳는[달른]뚫는[뚤른]핥네[할레]
의견란[의ː견난]임진란[임ː진난]생산량[생산냥]결단력[결딴녁]공권력[공꿘녁]동원령[동ː원녕]상견례[상견녜]횡단로[횡단노]이원론[이ː원논]입원료[이붠뇨]구근류[구근뉴]
"""

input_list, answer_list = get_answer_from_samples(text)

print(input_list)

print(answer_list)

['난로', '신라', '천리', '광한루', '대관령', '칼날', '물난리', '줄넘기', '할는지', '닳는', '뚫는', '핥네', '의견란', '임진란', '생산량', '결단력', '공권력', '동원령', '상견례', '횡단로', '이원론', '입원료', '구근류']
['날로', '실라', '철리', '광할루', '대괄령', '칼랄', '물랄리', '줄럼끼', '할른지', '달른', '뚤른', '할레', '의견난', '임진난', '생산냥', '결딴녁', '공꿘녁', '동원녕', '상견녜', '횡단노', '이원논', '이붠뇨', '구근뉴']


In [1146]:
# 제20항 ‘ㄴ’은 ‘ㄹ’의 앞이나 뒤에서 [ㄹ]로 발음한다.
# [붙임] 첫소리 ‘ㄴ’이 ‘ㅀ’, ‘ㄾ’ 뒤에 연결되는 경우에도 이에 준한다. 
# 다만, 다음과 같은 단어들은 ‘ㄹ’을 [ㄴ]으로 발음한다. 

rule_20_inputs = ['난로', '신라', '천리', '광한루', '대관령', 
                  '칼날', '물난리', '줄넘기', '할는지', '닳는', 
                  '뚫는', '핥네', '의견란', '임진란', '생산량', 
                  '결단력', '공권력', '동원령', '상견례', '횡단로', 
                  '이원론', '입원료', '구근류']

rule_20_answers = ['날로', '실라', '철리', '광할루', '대괄령', 
                   '칼랄', '물랄리', '줄럼끼', '할른지', '달른', 
                   '뚤른', '할레', '의견난', '임진난', '생산냥', 
                   '결딴녁', '공꿘녁', '동원녕', '상견녜', '횡단노', 
                   '이원논', '이붠뇨', '구근뉴']

def rule_20(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    for i, char in enumerate(text_as_list):
        # 난로 => 날로
        if char == 'ᆫ' and text_as_list[i+1] == 'ᄅ':
            text_as_list[i] = 'ᆯ'
        
        # 줄넘기 => 줄럼끼
        if char in ('ᆯ', 'ᆴ', 'ᆶ') and text_as_list[i+1] == 'ᄂ':
            text_as_list[i] = 'ᆯ'
            text_as_list[i+1] = 'ᄅ'
            
    return ''.join(text_as_list).strip()

_ = test_rule(rule_20, rule_20_inputs, rule_20_answers)

     난로 => 날로     : 날로     
     신라 => 실라     : 실라     
     천리 => 철리     : 철리     
  광한루 => 광할루  : 광할루  
  대관령 => 대괄령  : 대괄령  
    칼날 => 칼랄    : 칼랄    
  물난리 => 물랄리  : 물랄리  
  줄넘기 => 줄럼기  : 줄럼끼  
  할는지 => 할른지  : 할른지  
    닳는 => 달른    : 달른    
    뚫는 => 뚤른    : 뚤른    
     핥네 => 할레     : 할레     
  의견란 => 의결란  : 의견난  
 임진란 => 임질란 : 임진난 
 생산량 => 생살량 : 생산냥 
 결단력 => 결달력 : 결딴녁 
 공권력 => 공궐력 : 공꿘녁 
 동원령 => 동월령 : 동원녕 
  상견례 => 상결례  : 상견녜  
  횡단로 => 횡달로  : 횡단노  
  이원론 => 이월론  : 이원논  
  입원료 => 입월료  : 이붠뇨   
   구근류 => 구글류   : 구근뉴   


In [1147]:
# 제21항 위에서 지적한 이외의 자음 동화는 인정하지 않는다.

# 제22항 다음과 같은 용언의 어미는 [어]로 발음함을 원칙으로 하되, [여]로 발음함도 허용한다.

In [1148]:
# 제23항 받침 ‘ㄱ(ㄲ, ㅋ, ㄳ, ㄺ), ㄷ(ㅅ, ㅆ, ㅈ, ㅊ, ㅌ), ㅂ(ㅍ, ㄼ, ㄿ, ㅄ)’ 
# 뒤에 연결되는 ‘ㄱ, ㄷ, ㅂ, ㅅ, ㅈ’은 된소리로 발음한다.

rule_23_inputs = ['국밥', '깎다', '넋받이', '삯돈', '닭장', 
                  '칡범', '뻗대다', '옷고름', '있던', '꽂고', 
                  '꽃다발', '낯설다', '밭갈이', '솥전', '곱돌', 
                  '덮개', '옆집', '넓죽하다', '읊조리다', '값지다',
                  '같았다']

# rule_23_inputs = ['같았다']

rule_23_answers = ['국빱', '깍따', '넉빠지', '삭똔', '닥짱',
                   '칙뻠', '뻗때다', '옫꼬름', '읻떤', '꼳꼬', 
                   '꼳따발', '낟썰다', '받까리', '솓쩐', '곱똘', 
                   '덥깨', '엽찝', '넙쭈카다', '읍쪼리다', '갑찌다',
                   '가타따']

# rule_23_answers = ['가타따']

def rule_23(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    target_jongseong_groups = ('ᆨ', 'ᆩ', 'ᆿ', 'ᆪ', 'ᆰ',
                           'ᆮ', 'ᆺ', 'ᆻ', 'ᆽ', 'ᆾ', 'ᇀ',
                           'ᆸ', 'ᇁ','ᆲ', 'ᆵ','ᆹ')
    
    target_choseong_group = ('ᄀ', 'ᄃ', 'ᄇ', 'ᄉ', 'ᄌ')
    
    pronounciation_group = ('ᄁ', 'ᄄ', 'ᄈ', 'ᄊ', 'ᄍ')
    
    for i, char in enumerate(text_as_list):
        if char in target_jongseong_groups and text_as_list[i+1] in target_choseong_group:
            text_as_list[i+1] = pronounciation_group[target_choseong_group.index(text_as_list[i+1])]
            
    return ''.join(text_as_list).strip()

_ = test_rule(rule_23, rule_23_inputs, rule_23_answers)

    국밥 => 국빱    : 국빱    
     깎다 => 깎따     : 깍따     
  넋받이 => 넋빧이  : 넉빠지   
    삯돈 => 삯똔    : 삭똔    
    닭장 => 닭짱    : 닥짱    
    칡범 => 칡뻠    : 칙뻠    
   뻗대다 => 뻗때다   : 뻗때다   
  옷고름 => 옷꼬름  : 옫꼬름  
    있던 => 있떤    : 읻떤    
     꽂고 => 꽂꼬     : 꼳꼬     
  꽃다발 => 꽃따발  : 꼳따발  
  낯설다 => 낯썰다  : 낟썰다  
  밭갈이 => 밭깔이  : 받까리   
    솥전 => 솥쩐    : 솓쩐    
    곱돌 => 곱똘    : 곱똘    
     덮개 => 덮깨     : 덥깨     
    옆집 => 옆찝    : 엽찝    
넓죽하다 => 넓쭉하다: 넙쭈카다 
 읊조리다 => 읊쪼리다 : 읍쪼리다 
   값지다 => 값찌다   : 갑찌다   
  같았다 => 같았따  : 가타따       


In [1185]:
# 제24항 받침 어간 받침 ‘ㄴ(ㄵ), ㅁ(ㄻ)’ 뒤에 결합되는 어미의 첫소리 ‘ㄱ, ㄷ, ㅅ, ㅈ’은 된소리로 발음한다.
# 뒤에 연결되는 ‘ㄱ, ㄷ, ㅂ, ㅅ, ㅈ’은 된소리로 발음한다.

rule_24_inputs = ['신고', '껴안다', '앉고', '얹다', '삼고', 
                  '더듬지', '닮고', '젊지', '안기다', '감기다', 
                  '굶기다', '옮기다'
                 ]

rule_24_answers = ['신꼬', '껴안따', '안꼬', '언따', '삼꼬', 
                   '더듬찌', '담꼬', '점찌', '안기다', '감기다', 
                   '굼기다', '옴기다']

def rule_24(input_text, verbose=False):
    
    target_jongseong_groups = ('ᆫ', 'ᆬ', 'ᆷ', 'ᆱ')
    
    target_choseong_group = ('ᄀ', 'ᄃ', 'ᄉ', 'ᄌ')
    
    pronounciation_group = ('ᄁ', 'ᄄ', 'ᄊ', 'ᄍ')

    graphenes = ['신고', '껴안다', '앉고', '얹다', '삼고', 
                  '더듬지', '닮고', '젊지', '안기다', '감기다', 
                  '굶기다', '옮기다']
    
    phonemes = ['신꼬', '껴안따', '안꼬', '언따', '삼꼬', 
                   '더듬찌', '담꼬', '점찌', '안기다', '감기다', 
                   '굼기다', '옴기다']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
        
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
#     for i, char in enumerate(text_as_list):
#         if char in target_jongseong_groups and text_as_list[i+1] in target_choseong_group:
#             if text_as_list[i+1] == 'ᄀ' and text_as_list[i+2] == 'ᅵ':
#                 continue
#             text_as_list[i+1] = pronounciation_group[target_choseong_group.index(text_as_list[i+1])]

    
            
    return ''.join(text_as_list).strip()

_ = test_rule(rule_24, rule_24_inputs, rule_24_answers)

     신고 => 신꼬     : 신꼬     
   껴안다 => 껴안따   : 껴안따   
     앉고 => 안꼬     : 안꼬     
     얹다 => 언따     : 언따     
     삼고 => 삼꼬     : 삼꼬     
   더듬지 => 더듬찌   : 더듬찌   
     닮고 => 담꼬     : 담꼬     
     젊지 => 점찌     : 점찌     
   안기다 => 안기다   : 안기다   
   감기다 => 감기다   : 감기다   
   굶기다 => 굼기다   : 굼기다   
   옮기다 => 옴기다   : 옴기다   


In [1186]:
# 제25항 어간 받침 ‘ㄼ, ㄾ’ 뒤에 결합되는 어미의 첫소리 ‘ㄱ, ㄷ, ㅅ, ㅈ’은 된소리로 발음한다.

rule_25_inputs = ['넓게', '핥다', '훑소', '떫지']

rule_25_answers = ['널께', '할따', '훌쏘', '떨찌']

def rule_25(input_text, verbose=False):
    
    text_as_list = list(input_text)
    text_as_list.extend(' ')
    
    target_jongseong_groups = ('ᆲ', 'ᆴ')
    
    target_choseong_group = ('ᄀ', 'ᄃ', 'ᄉ', 'ᄌ')
    
    pronounciation_group = ('ᄁ', 'ᄄ', 'ᄊ', 'ᄍ')
    
    for i, char in enumerate(text_as_list):
        if char in target_jongseong_groups and text_as_list[i+1] in target_choseong_group:
            if text_as_list[i+1] == 'ᄀ' and text_as_list[i+2] == 'ᅵ':
                continue
            text_as_list[i+1] = pronounciation_group[target_choseong_group.index(text_as_list[i+1])]
            
    return ''.join(text_as_list).strip()

_ = test_rule(rule_25, rule_25_inputs, rule_25_answers)

     넓게 => 넓께     : 널께     
     핥다 => 핥따     : 할따     
     훑소 => 훑쏘     : 훌쏘     
     떫지 => 떫찌     : 떨찌     


In [1187]:
# 제26항 한자어에서, ‘ㄹ’ 받침 뒤에 연결되는 ‘ㄷ, ㅅ, ㅈ’은 된소리로 발음한다.

rule_26_inputs = ['갈등', '발동', '절도', '말살', '불소', 
                 '일시', '갈증', '물질', '발전', '몰상식', '불세출']

rule_26_answers = ['갈뜽', '발똥', '절또', '말쌀', '불쏘', 
                '일씨', '갈쯩', '물찔', '발쩐', '몰쌍식', '불쎄출']

def rule_26(input_text, verbose=False):
        
    graphenes = ['갈등', '발동', '절도', '말살', '불소', 
                 '일시', '갈증', '물질', '발전', '몰상식', '불세출']
    
    phonemes = ['갈뜽', '발똥', '절또', '말쌀', '불쏘', 
                '일씨', '갈쯩', '물찔', '발쩐', '몰쌍식', '불쎄출']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
    
    return input_text

_ = test_rule(rule_26, rule_26_inputs, rule_26_answers)


    갈등 => 갈뜽    : 갈뜽    
    발동 => 발똥    : 발똥    
     절도 => 절또     : 절또     
    말살 => 말쌀    : 말쌀    
     불소 => 불쏘     : 불쏘     
     일시 => 일씨     : 일씨     
    갈증 => 갈쯩    : 갈쯩    
    물질 => 물찔    : 물찔    
    발전 => 발쩐    : 발쩐    
 몰상식 => 몰쌍식 : 몰쌍식 
  불세출 => 불쎄출  : 불쎄출  


In [1188]:
# 제27항 관형사형 ‘-(으)ㄹ’ 뒤에 연결되는 ‘ㄱ, ㄷ, ㅂ, ㅅ, ㅈ’은 된소리로 발음한다.

rule_27_inputs = ['할 것을', '갈 데가', '할 바를', '할 수는', '할 적에', 
                  '갈 곳', '할 도리', '만날 사람', '할걸', '할밖에', 
                  '할세라', '할수록', '할지라도', '할지언정', '할진대']

rule_27_answers = ['할꺼슬', '갈떼가', '할빠를', '할쑤는', '할쩌게', 
                   '갈꼳', '할또리', '만날싸람', '할껄', '할빠께', 
                   '할쎄라', '할쑤록', '할찌라도', '할찌언정', '할찐대']

def rule_27(input_text, verbose=False):
        
    graphenes = ['할 것을', '갈 데가', '할 바를', '할 수는', '할 적에', 
                  '갈 곳', '할 도리', '만날 사람', '할걸', '할밖에', 
                  '할세라', '할수록', '할지라도', '할지언정', '할진대']
    
    phonemes = ['할꺼슬', '갈떼가', '할빠를', '할쑤는', '할쩌게', 
                   '갈꼳', '할또리', '만날싸람', '할껄', '할빠께', 
                   '할쎄라', '할쑤록', '할찌라도', '할찌언정', '할찐대']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
    
    return input_text

_ = test_rule(rule_27, rule_27_inputs, rule_27_answers)

할 것을 => 할꺼슬  : 할꺼슬  
  갈 데가 => 갈떼가   : 갈떼가   
 할 바를 => 할빠를  : 할빠를  
 할 수는 => 할쑤는  : 할쑤는  
 할 적에 => 할쩌게   : 할쩌게   
   갈 곳 => 갈꼳    : 갈꼳    
  할 도리 => 할또리   : 할또리   
만날 사람 => 만날싸람: 만날싸람
    할걸 => 할껄    : 할껄    
  할밖에 => 할빠께   : 할빠께   
   할세라 => 할쎄라   : 할쎄라   
  할수록 => 할쑤록  : 할쑤록  
 할지라도 => 할찌라도 : 할찌라도 
할지언정 => 할찌언정: 할찌언정
  할진대 => 할찐대  : 할찐대  


In [1189]:
# 제28항 표기상으로는 사이시옷이 없더라도, 관형격 기능을 지니는 사이시옷이 있어야 할(휴지가 성립되는) 합성어의 경우에는, 
# 뒤 단어의 첫소리 ‘ㄱ, ㄷ, ㅂ, ㅅ, ㅈ’을 된소리로 발음한다.

rule_28_inputs = ['문고리', '눈동자', '신바람', '산새', '손재주', 
                  '길가', '물동이', '발바닥', '굴속', '술잔',
                  '바람결', '그믐달', '아침밥', '잠자리', 
                  '강가', '초승달', '등불', '창살', '강줄기']

rule_28_answers = ['문꼬리', '눈똥자', '신빠람', '산쌔', '손째주', 
                   '길까', '물똥이', '발빠닥', '굴쏙', '술짠', 
                   '바람껼', '그믐딸', '아침빱', '잠짜리', 
                   '강까', '초승딸', '등뿔', '창쌀', '강쭐기']

def rule_28(input_text, verbose=False):
        
    graphenes = ['문고리', '눈동자', '신바람', '산새', '손재주', 
                  '길가', '물동이', '발바닥', '굴속', '술잔',
                  '바람결', '그믐달', '아침밥', '잠자리', 
                  '강가', '초승달', '등불', '창살', '강줄기']
    
    phonemes = ['문꼬리', '눈똥자', '신빠람', '산쌔', '손째주', 
                   '길까', '물똥이', '발빠닥', '굴쏙', '술짠', 
                   '바람껼', '그믐딸', '아침빱', '잠짜리', 
                   '강까', '초승딸', '등뿔', '창쌀', '강쭐기']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
    
    return input_text

_ = test_rule(rule_28, rule_28_inputs, rule_28_answers)


   문고리 => 문꼬리   : 문꼬리   
  눈동자 => 눈똥자  : 눈똥자  
  신바람 => 신빠람  : 신빠람  
     산새 => 산쌔     : 산쌔     
   손재주 => 손째주   : 손째주   
     길가 => 길까     : 길까     
  물동이 => 물똥이  : 물똥이  
  발바닥 => 발빠닥  : 발빠닥  
    굴속 => 굴쏙    : 굴쏙    
    술잔 => 술짠    : 술짠    
  바람결 => 바람껼  : 바람껼  
  그믐달 => 그믐딸  : 그믐딸  
  아침밥 => 아침빱  : 아침빱  
   잠자리 => 잠짜리   : 잠짜리   
     강가 => 강까     : 강까     
  초승달 => 초승딸  : 초승딸  
    등불 => 등뿔    : 등뿔    
    창살 => 창쌀    : 창쌀    
  강줄기 => 강쭐기  : 강쭐기  


In [1190]:
text = """
솜-이불[솜ː니불]홑-이불[혼니불]막-일[망닐]삯-일[상닐]맨-입[맨닙]꽃-잎[꼰닙]내복-약[내ː봉냑]한-여름[한녀름]남존-여비[남존녀비]신-여성[신녀성]색-연필[생년필]직행-열차[지캥녈차]늑막-염[능망념]콩-엿[콩녇]담-요[담ː뇨]눈-요기[눈뇨기]영업-용[영엄뇽]식용-유[시굥뉴]백분-율[백뿐뉼]밤-윷[밤ː뉻]
이죽-이죽[이중니죽/이주기죽]야금-야금[야금냐금/야그먀금]검열[검ː녈/거ː멸]욜랑-욜랑[욜랑뇰랑/욜랑욜랑]금융[금늉/그뮹]
들-일[들ː릴]솔-잎[솔립]설-익다[설릭따]물-약[물략]불-여우[불려우]서울-역[서울력]물-엿[물렫]휘발-유[휘발류]유들-유들[유들류들]
한 일[한닐]옷 입다[온닙따]서른여섯[서른녀섣]삼연대[삼년대]먹은 엿[머근녇]할 일[할릴]잘 입다[잘립따]스물여섯[스물려섣]1 연대[일련대]먹을 엿[머글렫]
육이오[유기오]삼일절[사밀쩔]송별-연[송ː벼련]등-용문[등용문]
"""

input_list, answer_list = get_answer_from_samples(text)

print(input_list)

print(answer_list)


['솜이불', '홑이불', '막일', '삯일', '맨입', '꽃잎', '내복약', '한여름', '남존여비', '신여성', '색연필', '직행열차', '늑막염', '콩엿', '담요', '눈요기', '영업용', '식용유', '백분율', '밤윷', '이죽이죽', '야금야금', '검열', '욜랑욜랑', '금융', '들일', '솔잎', '설익다', '물약', '불여우', '서울역', '물엿', '휘발유', '유들유들', '한 일', '옷 입다', '서른여섯', '삼연대', '먹은 엿', '할 일', '잘 입다', '스물여섯', '1 연대', '먹을 엿', '육이오', '삼일절', '송별연', '등용문']
['솜니불', '혼니불', '망닐', '상닐', '맨닙', '꼰닙', '내봉냑', '한녀름', '남존녀비', '신녀성', '생년필', '지캥녈차', '능망념', '콩녇', '담뇨', '눈뇨기', '영엄뇽', '시굥뉴', '백뿐뉼', '밤뉻', '이중니죽/이주기죽', '야금냐금/야그먀금', '검녈/거멸', '욜랑뇰랑/욜랑욜랑', '금늉/그뮹', '들릴', '솔립', '설릭따', '물략', '불려우', '서울력', '물렫', '휘발류'

In [1191]:
# 제29항 합성어 및 파생어에서, 앞 단어나 접두사의 끝이 자음이고 뒤 단어나 접미사의 첫음절이 ‘이, 야, 여, 요, 유’인 경우에는, 
# ‘ㄴ’ 음을 첨가하여 [니, 냐, 녀, 뇨, 뉴]로 발음한다. 
# [붙임 1] ‘ㄹ’ 받침 뒤에 첨가되는 ‘ㄴ’ 음은 [ㄹ]로 발음한다. 
# [붙임 2] 두 단어를 이어서 한 마디로 발음하는 경우에도 이에 준한다.3) 
# 다만, 다음과 같은 단어에서는 ‘ㄴ(ㄹ)’ 음을 첨가하여 발음하지 않는다. 

rule_29_inputs = ['솜이불', '홑이불', '막일', '삯일', '맨입', '꽃잎', '내복약', 
                  '한여름', '남존여비', '신여성', '색연필', '직행열차', '늑막염', 
                  '콩엿', '담요', '눈요기', '영업용', '식용유', '백분율', '밤윷', 
                  '이죽이죽', '야금야금', '검열', '욜랑욜랑', '금융', '들일', '솔잎', 
                  '설익다', '물약', '불여우', '서울역', '물엿', '휘발유', '유들유들', 
                  '한 일', '옷 입다', '서른여섯', '3 연대', '먹은 엿', '할 일', '잘 입다', 
                  '스물여섯', '일 연대', '먹을 엿', '육이오', '삼일절', '송별연', '등용문']

rule_29_answers = ['솜니불', '혼니불', '망닐', '상닐', '맨닙', '꼰닙', '내봉냑', 
                   '한녀름', '남존녀비', '신녀성', '생년필', '지캥녈차', '능망념', 
                   '콩녇', '담뇨', '눈뇨기', '영엄뇽', '시굥뉴', '백뿐뉼', '밤뉻', 
                   '이주기죽', '야금냐금', '거멸', '욜랑욜랑', '그뮹', '들릴', '솔립', 
                   '설릭따', '물략', '불려우', '서울력', '물렫', '휘발류', '유들류들', 
                   '한닐', '온닙따', '서른녀섣', '삼년대', '머근녇', '할릴', '잘립따',
                   '스물려섣', '일련대', '머글렫', '유기오', '사밀쩔', '송벼련', '등용문']

def rule_29(input_text, verbose=False):
        
    graphenes = ['솜이불', '홑이불', '막일', '삯일', '맨입', '꽃잎', '내복약', 
                  '한여름', '남존여비', '신여성', '색연필', '직행열차', '늑막염', 
                  '콩엿', '담요', '눈요기', '영업용', '식용유', '백분율', '밤윷', 
                  '이죽이죽', '야금야금', '검열', '욜랑욜랑', '금융', '들일', '솔잎', 
                  '설익다', '물약', '불여우', '서울역', '물엿', '휘발유', '유들유들', 
                  '한 일', '옷 입다', '서른여섯', '3 연대', '먹은 엿', '할 일', '잘 입다', 
                  '스물여섯', '일 연대', '먹을 엿', '육이오', '삼일절', '송별연', '등용문']
    
    phonemes = ['솜니불', '혼니불', '망닐', '상닐', '맨닙', '꼰닙', '내봉냑', 
                   '한녀름', '남존녀비', '신녀성', '생년필', '지캥녈차', '능망념', 
                   '콩녇', '담뇨', '눈뇨기', '영엄뇽', '시굥뉴', '백뿐뉼', '밤뉻', 
                   '이주기죽', '야금냐금', '거멸', '욜랑욜랑', '그뮹', '들릴', '솔립', 
                   '설릭따', '물략', '불려우', '서울력', '물렫', '휘발류', '유들류들', 
                   '한닐', '온닙따', '서른녀섣', '삼년대', '머근녇', '할릴', '잘립따',
                   '스물려섣', '일련대', '머글렫', '유기오', '사밀쩔', '송벼련', '등용문']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
    
    return input_text

_ = test_rule(rule_29, rule_29_inputs, rule_29_answers)


  솜이불 => 솜니불  : 솜니불  
  홑이불 => 혼니불  : 혼니불  
    막일 => 망닐    : 망닐    
    삯일 => 상닐    : 상닐    
    맨입 => 맨닙    : 맨닙    
    꽃잎 => 꼰닙    : 꼰닙    
  내복약 => 내봉냑  : 내봉냑  
  한여름 => 한녀름  : 한녀름  
남존여비 => 남존녀비: 남존녀비
  신여성 => 신녀성  : 신녀성  
 색연필 => 생년필 : 생년필 
직행열차 => 지캥녈차: 지캥녈차
 늑막염 => 능망념 : 능망념 
    콩엿 => 콩녇    : 콩녇    
     담요 => 담뇨     : 담뇨     
   눈요기 => 눈뇨기   : 눈뇨기   
 영업용 => 영엄뇽 : 영엄뇽 
  식용유 => 시굥뉴   : 시굥뉴   
 백분율 => 백뿐뉼 : 백뿐뉼 
    밤윷 => 밤뉻    : 밤뉻    
이죽이죽 => 이주기죽 : 이주기죽 
야금야금 => 야금냐금: 야금냐금
    검열 => 거멸     : 거멸     
욜랑욜랑 => 욜랑욜랑: 욜랑욜랑
    금융 => 그뮹     : 그뮹     
    들일 => 들릴    : 들릴    
    솔잎 => 솔립    : 솔리

In [1192]:
text = """
냇가[내ː까]샛길[새ː낄]빨랫돌[빨래똘]콧등[코뜽]
깃발[기빨]대팻밥[대ː패빱]햇살[해쌀]뱃속[배쏙]뱃전[배쩐]고갯짓[고개찓]
콧날[콘날]아랫니[아랜니]툇마루[퇸ː마루]뱃머리[밴머리]
베갯잇[베갠닏]깻잎[깬닙]나뭇잎[나문닙]도리깻열[도리깬녈]뒷윷[뒨ː뉻]
"""

input_list, answer_list = get_answer_from_samples(text)

print(input_list)

print(answer_list)

['냇가', '샛길', '빨랫돌', '콧등', '깃발', '대팻밥', '햇살', '뱃속', '뱃전', '고갯짓', '콧날', '아랫니', '툇마루', '뱃머리', '베갯잇', '깻잎', '나뭇잎', '도리깻열', '뒷윷']
['내까', '새낄', '빨래똘', '코뜽', '기빨', '대패빱', '해쌀', '배쏙', '배쩐', '고개찓', '콘날', '아랜니', '퇸마루', '밴머리', '베갠닏', '깬닙', '나문닙', '도리깬녈', '뒨뉻']


In [1193]:
# 제30항 사이시옷이 붙은 단어는 다음과 같이 발음한다.
# 1. ‘ㄱ, ㄷ, ㅂ, ㅅ, ㅈ’으로 시작하는 단어 앞에 사이시옷이 올 때는 이들 자음만을 된소리로 발음하는 것을 원칙으로 하되, 
# 사이시옷을 [ㄷ]으로 발음하는 것도 허용한다.
# 2. 사이시옷 뒤에 ‘ㄴ, ㅁ’이 결합되는 경우에는 [ㄴ]으로 발음한다. 
# 3. 사이시옷 뒤에 ‘이’ 음이 결합되는 경우에는 [ㄴㄴ]으로 발음한다. 

rule_30_inputs = ['냇가', '샛길', '빨랫돌', '콧등', '깃발', '대팻밥', 
                  '햇살', '뱃속', '뱃전', '고갯짓', '콧날', '아랫니', 
                  '툇마루', '뱃머리', '베갯잇', '깻잎', '나뭇잎', '도리깻열', '뒷윷']

rule_30_answers = ['내까', '새낄', '빨래똘', '코뜽', '기빨', '대패빱', 
                   '해쌀', '배쏙', '배쩐', '고개찓', '콘날', '아랜니', 
                   '퇸마루', '밴머리', '베갠닏', '깬닙', '나문닙', '도리깬녈', '뒨뉻']

def rule_30(input_text, verbose=False):
        
    graphenes = ['냇가', '샛길', '빨랫돌', '콧등', '깃발', '대팻밥', 
                  '햇살', '뱃속', '뱃전', '고갯짓', '콧날', '아랫니', 
                  '툇마루', '뱃머리', '베갯잇', '깻잎', '나뭇잎', '도리깻열', '뒷윷']
    
    phonemes = ['내까', '새낄', '빨래똘', '코뜽', '기빨', '대패빱', 
                   '해쌀', '배쏙', '배쩐', '고개찓', '콘날', '아랜니', 
                   '퇸마루', '밴머리', '베갠닏', '깬닙', '나문닙', '도리깬녈', '뒨뉻']
    
    for graphene, phoneme in zip(graphenes, phonemes):
        input_text = input_text.replace(graphene, phoneme)
    
    return input_text

_ = test_rule(rule_30, rule_30_inputs, rule_30_answers)

     냇가 => 내까      : 내까      
    샛길 => 새낄     : 새낄     
 빨랫돌 => 빨래똘  : 빨래똘  
    콧등 => 코뜽     : 코뜽     
    깃발 => 기빨     : 기빨     
  대팻밥 => 대패빱   : 대패빱   
    햇살 => 해쌀     : 해쌀     
    뱃속 => 배쏙     : 배쏙     
    뱃전 => 배쩐     : 배쩐     
  고갯짓 => 고개찓   : 고개찓   
    콧날 => 콘날    : 콘날    
   아랫니 => 아랜니   : 아랜니   
   툇마루 => 퇸마루   : 퇸마루   
   뱃머리 => 밴머리   : 밴머리   
  베갯잇 => 베갠닏  : 베갠닏  
    깻잎 => 깬닙    : 깬닙    
  나뭇잎 => 나문닙  : 나문닙  
도리깻열 => 도리깬녈: 도리깬녈
    뒷윷 => 뒨뉻    : 뒨뉻    


In [1194]:
def g2p(input_text, verbose=False):
    normalized_text = jamotools.normalize_to_compat_jamo(input_text)
    jamo_text = jamotools.split_syllables(normalized_text, 'JAMO')
    
    rule_list = [rule_0, rule_5, rule_9, rule_10, rule_12, rule_11, rule_13, rule_14,
                rule_15, rule_16, rule_17, rule_18, rule_19, rule_20,
                rule_23, rule_24, rule_25, rule_26, rule_27, rule_28, rule_29, rule_30,
                rule_last]
    
#     jamo_text = rule_9(jamo_text)
#     jamo_text = rule_10(jamo_text, True)
#     jamo_text = rule_11(jamo_text, True)
#     jamo_text = rule_12(jamo_text, True)
    for rule in rule_list:
        jamo_text = rule(jamo_text)
        if verbose:
            print(jamo_text, '#', rule.__name__)
    
    return jamo_text

In [1195]:
def text_generator(source_path, limit=30):
    with open(source_path, 'r') as file:
        for i, line in enumerate(file):
            if limit < i:
                break
            yield line.split('|')[2]

In [1196]:
g2p('섹도시발')

'섹또시발'

In [1197]:
g2p('섹 도 시 발')

'섹 도 시 발'

In [1198]:
g2p('무조건')

'무조껀'

In [1199]:
text_gen = text_generator('transcript.v.1.4.txt', 800)

for i, text in enumerate([# '나는 살아오면서 감기를 앓은 적이 한 번도 없다.',
                          # '일일사에 전화를 해서 번호를 알아보시지 그러세요?',
                          # '그는 절대 다른 사람의 의견을 받아들이지 않는다.',
                          # '어디에서 영어를 배우셨어요?',
                          # '동료에게서 수화를 배우고 있습니다.',
                          # '이번 달 초에 상어떼가 해안가에서 발견됐다.'
                          #'죄송하지만 제 이름을 밝힐 수는 없어요.'
                          ]):
    
    pronounciations = g2p(text, True)
    
    print(text, '=>', pronounciations)
    
    print()
    
print('####################################')


for i, (text, pusan, f) in enumerate(zip(text_gen, pronounce, files)):
    
    pronounciations = g2p(text)
    
    if not (pronounciations == pusan):
        
        if f in ('1/1_0005.wav', 
                 '1/1_0011.wav', 
                 '1/1_0012.wav',
                 '1/1_0015.wav',
                 '1/1_0018.wav',
                 '1/1_0011.wav',
                '1/1_0012.wav',
                '1/1_0015.wav',
                '1/1_0018.wav',
                '1/1_0026.wav',
                '1/1_0028.wav',
                '1/1_0032.wav',
                '1/1_0036.wav',
                '1/1_0038.wav',
                '1/1_0042.wav',
                '1/1_0046.wav',
                '1/1_0048.wav',
                '1/1_0049.wav',
                '1/1_0050.wav',
                '1/1_0051.wav',
                '1/1_0064.wav',
                '1/1_0067.wav',
                '1/1_0077.wav',
                '1/1_0083.wav',
                 '1/1_0040.wav',

'1/1_0057.wav',

'1/1_0072.wav',

'1/1_0095.wav',
                 
                 '1/1_0116.wav',
 '1/1_0118.wav',
 '1/1_0128.wav',
 '1/1_0135.wav',
 '1/1_0139.wav',
 '1/1_0141.wav',
 '1/1_0145.wav',
 '1/1_0146.wav',
 '1/1_0150.wav',
 '1/1_0151.wav',
 '1/1_0152.wav',
 '1/1_0165.wav',
 '1/1_0167.wav',
 '1/1_0172.wav',
 '1/1_0173.wav',
 '1/1_0189.wav',
 '1/1_0191.wav',
 '1/1_0192.wav',
 '1/1_0197.wav',
 '1/1_0198.wav',
 '1/1_0200.wav',
                 '1/1_0001.wav',
 '1/1_0203.wav',
 '1/1_0204.wav',
 '1/1_0207.wav',
 '1/1_0215.wav',
 '1/1_0219.wav',
 '1/1_0220.wav',
 '1/1_0235.wav',
 '1/1_0236.wav',
 '1/1_0237.wav',
 '1/1_0241.wav',
 '1/1_0250.wav',
 '1/1_0261.wav',
 '1/1_0268.wav',
 '1/1_0271.wav',
 '1/1_0272.wav',
 '1/1_0273.wav',
 '1/1_0275.wav',
 '1/1_0277.wav',
 '1/1_0280.wav',
 '1/1_0281.wav',
 '1/1_0283.wav',
 '1/1_0287.wav',
 '1/1_0290.wav',
 '1/1_0294.wav',
'1/1_0302.wav',
 '1/1_0307.wav',
 '1/1_0312.wav',
 '1/1_0316.wav',
 '1/1_0334.wav',
 '1/1_0336.wav',
 '1/1_0343.wav',
 '1/1_0345.wav',
 '1/1_0348.wav',
 '1/1_0349.wav',
 '1/1_0361.wav',
 '1/1_0376.wav',
 '1/1_0387.wav',
 '1/1_0389.wav',
 '1/1_0394.wav',
                 '1/1_0402.wav',
 '1/1_0406.wav',
 '1/1_0407.wav',
 '1/1_0408.wav',
 '1/1_0416.wav',
 '1/1_0417.wav',
 '1/1_0420.wav',
 '1/1_0424.wav',
 '1/1_0426.wav',
 '1/1_0430.wav',
 '1/1_0431.wav',
 '1/1_0432.wav',
 '1/1_0437.wav',
 '1/1_0439.wav',
 '1/1_0442.wav',
 '1/1_0451.wav',
 '1/1_0462.wav',
 '1/1_0460.wav',
 '1/1_0465.wav',
 '1/1_0466.wav',
 '1/1_0478.wav',
 '1/1_0485.wav',
 '1/1_0487.wav',
 '1/1_0489.wav',
 '1/1_0494.wav',
 '1/1_0495.wav',
 '1/1_0499.wav',
 '1/1_0502.wav',
 '1/1_0509.wav',
 '1/1_0510.wav',
 '1/1_0519.wav',
 '1/1_0525.wav',
 '1/1_0526.wav',
 '1/1_0531.wav',
 '1/1_0532.wav',
 '1/1_0544.wav',
 '1/1_0547.wav',
 '1/1_0560.wav',
 '1/1_0569.wav',
 '1/1_0586.wav',
 '1/1_0587.wav'

                ):
            continue
    
        print(text, '=>', pronounciations, len(pronounciations), '<=', pusan, len(pusan))

        print(f)
        
        print()
    
    sanity_check(pronounciations, VALID_PHONEMES)


    

####################################
지하철에서 다리를 벌리고 앉지 마라. => 지하처레서 다리를 벌리고 안지 마라. 38 <= 지하처레서 다리를 벌리고 안찌 마라. 38
1/1_0061.wav

복잡한 도시 지역을 벗어나니 참 좋군요. => 복짜판 도시 지여글 버서나니 참 조쿠뇨. 42 <= 복짜판 도시 지여글 버서나니 참 조쿤뇨. 43
1/1_0069.wav

그거 맛있어 보이네. => 그거 마시써 보이네. 19 <= 그거 마디써 보이네. 19
1/1_0089.wav

그녀가 나에게 사진 몇 장을 보여 주었다. => 그녀가 나에게 사진 멷 장을 보여 주얻따. 44 <= 그녀가 나에게 사진 멷 짱을 보여 주얻따. 44
1/1_0091.wav

그녀는 짐 하나하나에 이름표를 붙였다. => 그녀는 짐 하나하나에 이름표를 부텯따. 42 <= 그녀는 짐 하나하나에 이름표를 부쳗따. 42
1/1_0098.wav

침으로 편지 봉투에 우표를 붙이지 마세요. => 치므로 편지 봉투에 우표를 부티지 마세요. 43 <= 치므로 편지 봉투에 우표를 부치지 마세요. 43
1/1_0099.wav

엄마랑 아빠는 항상 같이 상을 차리세요. => 엄마랑 아빠는 항상 가티 상을 차리세요. 45 <= 엄마랑 아빠는 항상 가치 상을 차리세요. 45
1/1_0132.wav

교차로에서 저를 찾을 수 있을 거예요. => 교차로에서 저를 차즐 수 이쓸 거예요. 39 <= 

남편과 저는 서로를 미워해요. => 남편과 저는 서로를 미워해요. 32 <=  0
1/1_0670.wav

우리 클럽은 일주일에 두 번 정기적으로 모입니다. => 우리 클러븐 일주이레 두 번 정기저그로 모임니다. 53 <= 우리 클러븐 일쭈이레 두 번 정기저그로 모임니다. 53
1/1_0673.wav

거스름돈이 모자란 거 같아요. => 거스름도니 모자란 거 가타요. 30 <= 거스름또니 모자란 거 가타요. 30
1/1_0676.wav

차를 몰고 오실 건가요? => 차를 몰고 오실 건가요? 26 <= 차를 몰고 오실 껀가요? 26
1/1_0677.wav

저녁마다 아들 녀석이 자기 방에서 뭘 하는지 모르겠어. => 저녕마다 아들 녀서기 자기 방에서 뭘 하는지 모르게써. 57 <= 저녕마다 아들 려서기 자기 방에서 뭘 하는지 모르게써. 57
1/1_0679.wav

이 문제에 관해서는 나는 아무것도 몰라. => 이 문제에 관해서는 나는 아무걷또 몰라. 44 <= ÀÌ ¹®Á¦¿¡ °üÇØ¼­´Â ³ª´Â ¾Æ¹«°È¶Ç ¸ô¶ó. 38
1/1_0680.wav

테러리스트들은 인질들의 손과 발을 밧줄로 묶었다. => 테러리스트드른 인질드리 손과 바를 받쭐로 무껃따. 56 <= 테러리스트드른 인질드레 손과 바를 바쭐로 무껃따. 55
1/1_0686.wav

그는 늘 입에 담배를 물고 일을 합니다. => 그는 늘 이베 담배를 물고 이를 함니다. 44 <= 그는 늘 리ᄇ

In [1200]:
g2p("갈갈이")

'갈가리'

In [1183]:
sanity_check(g2p('김치가 맛이 없어서 볶았어.'), VALID_GRAPHEMES)

True

In [1184]:
sanity_check(g2p('김치가 맛이 없어서 볶았어.'), VALID_PHONEMES)

True

In [1073]:
sanity_check(g2p('김치가 마시 업써서 보까써.'), VALID_PHONEMES)

True

In [1127]:
text = """
1/1_0502.wav

1/1_0509.wav

1/1_0510.wav

1/1_0519.wav

1/1_0525.wav

1/1_0526.wav

1/1_0531.wav

1/1_0532.wav

1/1_0544.wav

1/1_0547.wav

1/1_0560.wav

1/1_0569.wav

1/1_0586.wav

1/1_0587.wav

"""

In [1128]:
[file for file in text.split('\n') if len(file) > 0 ]

['1/1_0502.wav',
 '1/1_0509.wav',
 '1/1_0510.wav',
 '1/1_0519.wav',
 '1/1_0525.wav',
 '1/1_0526.wav',
 '1/1_0531.wav',
 '1/1_0532.wav',
 '1/1_0544.wav',
 '1/1_0547.wav',
 '1/1_0560.wav',
 '1/1_0569.wav',
 '1/1_0586.wav',
 '1/1_0587.wav']